In [1]:
from flask import Flask, jsonify, abort, make_response, request
from flask_sqlalchemy import SQLAlchemy
from datetime import datetime

# Flask Config

In [2]:
user = 'root'
password = 'root'
host = 'localhost'
port = 3306
dbname = 'surabayapy'

db_uri = 'mysql+mysqldb://%s:%s@%s:%d/%s' % (user, password, host, port, dbname)

In [3]:
app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = db_uri
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = True
db = SQLAlchemy(app)

# Supporting Function

In [4]:
import hashlib
def hashit(text):
    text = text.encode('UTF-8')
    hash_object = hashlib.sha256(text)
    hex_dig = hash_object.hexdigest()
    return hex_dig

In [5]:
def vdir(obj):
    return [x for x in dir(obj) if not x.startswith('_')]

In [6]:
def savedb(data):
    db.session.add(data)
    db.session.commit()

In [7]:
def waktu(data):
    return datetime.strptime(data, '%d %b %Y %H:%M')

# Database

In [8]:
daftar_hadir = db.Table('daftar_hadir',
    db.Column('kegiatan',db.Integer, db.ForeignKey('kegiatan.id'), index=True),
    db.Column('peserta',db.Integer, db.ForeignKey('anggota.id'), index=True),
)

In [9]:
class Anggota(db.Model):
    __table_args__ = {'extend_existing': True}  
    
    id = db.Column(db.Integer, primary_key=True)
    nama = db.Column(db.String(80), nullable=False)
    email = db.Column(db.String(120), unique=True, nullable=False)
    telegram = db.Column(db.String(64), unique=True, nullable=False)
    github = db.Column(db.String(15), unique=True, nullable=True)
    key = db.Column(db.String(4), unique=True, nullable=True)
    cari_kerja = db.Column(db.Boolean, nullable=False)
    tanggal_terdaftar = db.Column(db.DateTime, default=datetime.utcnow)
    ikut_serta = db.relationship('Kegiatan', secondary='daftar_hadir', backref=db.backref('peserta', lazy='dynamic'))

    def __repr__(self):
        return '<Anggota %r>' % self.telegram

In [10]:
class Kegiatan(db.Model):
    __table_args__ = {'extend_existing': True} 
    
    id = db.Column(db.Integer, primary_key=True)
    tema =  db.Column(db.Text, nullable=False)
    pembicara = db.Column(db.Text,  nullable=False)
    tempat = db.Column(db.Text,  nullable=False)
    maps = db.Column(db.String(80), nullable=True)
    waktu = db.Column(db.DateTime, nullable=False)
    pendaftaran = db.Column(db.DateTime, nullable=False)

    def __repr__(self):
        return '<Kegiatan %r>' % self.tema

In [11]:
class Lowongan(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    pengirim = db.Column(db.String(80), nullable=False)
    judul = db.Column(db.String(80),  nullable=False)
    pesan = db.Column(db.Text, nullable=False)
    
    def __repr__(self):
        return '<Lowongan %r>' % self.judul

In [12]:
db.create_all()

# Fungsi Operasi

In [13]:
def registrasi_anggota(nama, email, telegram, github='', cari_kerja=False):
    try:
        anggota_baru = Anggota(nama=nama, email=email, telegram=telegram, github=github, cari_kerja=cari_kerja)
        savedb(anggota_baru)
        
        data= {
            'id' : anggota_baru.id,
            'nama' : anggota_baru.nama,
            'email' : anggota_baru.email,
            'telegram' : anggota_baru.telegram,
            'github' : anggota_baru.github,
            'cari_kerja' : anggota_baru.cari_kerja
        }
        return {'status': 'success', 'data': data}, 201
    
    except:
        db.session.rollback()
        return {'status': 'failed', 'data': 'Username, email, telegram atau github sudah digunakan'}, 400

In [14]:
def buat_kegiatan(tema, pembicara, tempat, maps, waktuacara, pendaftaran):
    try:
        kegiatan_baru = Kegiatan(tema = tema, pembicara = pembicara, tempat = tempat, maps = maps, waktu = waktu(waktuacara), pendaftaran = waktu(pendaftaran))
        savedb(kegiatan_baru)
        data= {
            'id' : kegiatan_baru.id,
            'tema' : kegiatan_baru.tema,
            'pembicara' : kegiatan_baru.pembicara,
            'tempat' : kegiatan_baru.tempat,
            'maps' : kegiatan_baru.maps,
            'waktu' : kegiatan_baru.waktu,
            'pendaftaran' : kegiatan_baru.pendaftaran
        }
        
        return {'status': 'success', 'data': data}, 201
    except:
        db.session.rollback()
        return {'status': 'failed', 'data': 'Kegiatan tidak dapat didaftarkan'}, 400

In [15]:
def kegiatan_peserta(anggota_telegram):
    try:
        anggota = Anggota.query.filter_by(telegram=anggota_telegram).first()
        info_anggota =[]
        for kegiatan in anggota.ikut_serta:
            data = {
                'id': kegiatan.id,
                'tema': kegiatan.tema,
                'pembicara': kegiatan.pembicara,
                'tempat': kegiatan.tempat,
                'maps': kegiatan.maps,
                'waktu': kegiatan.waktu
            }
            info_anggota.append(data)
        return {'status': 'success', 'data': info_anggota}, 200
    except:
        return {'status': 'failed', 'data': 'Not found'}, 404

In [16]:
def mendaftar_kegiatan(anggota_telegram, idkegiatan):
    try:
        anggota = Anggota.query.filter_by(telegram=anggota_telegram).first()
        kegiatan = Kegiatan.query.filter_by(id=idkegiatan).first()
        kegiatan.peserta.append(anggota)
        db.session.commit()
        response = kegiatan_peserta(anggota_telegram)
        return response[0], 201
    except:
        return {'status': 'failed', 'data': 'Not found'}, 404

In [17]:
def list_kegiatan(mode, idkegiatan=0):
    try:
        if mode == 'tema':
            daftar_kegiatan = []
            for kegiatan in Kegiatan.query:
                data = {
                    'tema' : kegiatan.tema,
                    'id' : kegiatan.id
                }
                daftar_kegiatan.append(data)
            return {'status': 'success', 'data': daftar_kegiatan}, 200
        
        elif mode == 'peserta':
            kegiatan = Kegiatan.query.filter_by(id=idkegiatan).first()
            daftar_peserta = []
            for peserta in kegiatan.peserta:
                daftar_peserta.append(peserta.telegram)
            return {'status': 'success', 'data': daftar_peserta}, 200
        
    except:
        return {'status': 'failed', 'data': 'Not found'}, 404

# API

HTTP Method | Action | Examples | Fungsi yang digunakan
--- | --- | --- | ---
Post | Registrasi Anggota | /api/daftar-anggota/ | registrasi_anggota(nama, email, telegram, github, cari_kerja)
**Post** | **Membuat Kegiatan**| **/api/buat-kegiatan/** | **buat_kegiatan(tema, pembicara, tempat, maps, waktu, pendaftaran)**
Post | Mendaftar Kegiatan| /api/mendaftar-kegiatan/ | mendaftar_kegiatan(anggota_telegram, tema)
Get | List semua kegiatan selanjutnya| /api/daftar-kegiatan/ | list_kegiatan('tema')
Get | List semua peserta kegiatan| /api/peserta-kegiatan/id-kegiatan/ | list_kegiatan('peserta', id-kegiatan)
Get | List semua kegiatan peserta| /api/kegiatan/id-telegram/ | kegiatan_peserta(id-telegram)

**Bold** bisa diakses oleh admin saja

In [18]:
@app.route('/api/daftar-anggota/', methods=['POST'])
def api_daftar_anggota():
    if request.method == 'POST':
        parameter = ['nama', 'email', 'telegram', 'github']
        if not request.json or not set(parameter).issubset(request.json):
            response = {'status': 'failed', 'data': 'Bad Request'}, 400
        else:
            content = request.json
            response = registrasi_anggota(content["nama"], content["email"], content["telegram"], content["github"])
    return jsonify(response[0]), response[1]
    

@app.route('/api/buat-kegiatan/', methods=['POST'])
def api_buat_kegiatan():
    if request.method == 'POST':
        parameter = ['tema', 'pembicara', 'tempat', 'maps', 'waktu', 'pendaftaran']
        if not request.json or not set(parameter).issubset(request.json):
            response = {'status': 'failed', 'data': 'Bad Request'}, 400
        else:
            content = request.json
            response = buat_kegiatan(content["tema"], content["pembicara"], content["tempat"], content["maps"], content["waktu"], content["pendaftaran"])
    return jsonify(response[0]), response[1]
    

@app.route('/api/mendaftar-kegiatan/', methods=['POST'])
def api_mendaftar_kegiatan():
    if request.method == 'POST':
        parameter = ['anggota_telegram', 'idkegiatan']
        if not request.json or not set(parameter).issubset(request.json):
            response = {'status': 'failed', 'data': 'Bad Request'}, 400
        else:
            content = request.json
            response = mendaftar_kegiatan(content["anggota_telegram"], content["idkegiatan"])
    return jsonify(response[0]), response[1]
     

@app.route('/api/daftar-kegiatan/', methods=['GET'])
def api_kegiatan():
    if request.method == 'GET':
        response = list_kegiatan('tema')
    return jsonify(response[0]), response[1]
     

@app.route('/api/peserta-kegiatan/<int:idkegiatan>/', methods=['GET'])
def api_peserta_kegiatan(idkegiatan):
    if request.method == 'GET':
        response = list_kegiatan('peserta', idkegiatan)
    return jsonify(response[0]), response[1]
     

@app.route('/api/kegiatan/<idtelegram>/', methods=['GET'])
def api_kegiatan_idtelegram(idtelegram):
    if request.method == 'GET':
        response = kegiatan_peserta(idtelegram)
    return jsonify(response[0]), response[1]
     

In [19]:
@app.errorhandler(404)
def not_found(error):
    return make_response(jsonify({'error': 'Not  Found'}), 404)

@app.errorhandler(400)
def not_found(error):
    return make_response(jsonify({'error': 'Bad Requesst'}), 404)

In [20]:
if __name__ == '__main__':
    app.run(host= '0.0.0.0',debug=False)

 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [17/Apr/2018 00:21:47] "GET /api/kegiatan/tegarimansyah/ HTTP/1.1" 200 -


# Test Fungsi

In [19]:
registrasi_anggota('tegar','egar@imansyah.email', 'tegarimansyah', 'tegarimansyah', True)

({'data': {'cari_kerja': True,
   'email': 'egar@imansyah.email',
   'github': 'tegarimansyah',
   'id': 1,
   'nama': 'tegar',
   'telegram': 'tegarimansyah'},
  'status': 'success'},
 201)

In [30]:
buat_kegiatan('Combining Web, Bot and AI in One App with Python', 'Tegar Imansyah', 'Unair C', None, waktu('18 Apr 2018 18:00'), waktu('18 Apr 2018 17:00'))

({'data': {'id': 1,
   'maps': None,
   'pembicara': 'Tegar Imansyah',
   'pendaftaran': datetime.datetime(2018, 4, 18, 17, 0),
   'tema': 'Combining Web, Bot and AI in One App with Python',
   'tempat': 'Unair C',
   'waktu': datetime.datetime(2018, 4, 18, 18, 0)},
  'status': 'success'},
 201)

In [64]:
tema = 'Combining Web, Bot and AI in One App with Python'

({'data': ['Combining Web, Bot and AI in One App with Python'],
  'status': 'success'},
 201)

In [65]:
tegar = Anggota.query.filter_by(telegram='tegarimansyah').first()

In [67]:
kegiatan = Kegiatan.query.filter_by(tema=tema).first()

In [71]:
tegar.ikut_serta

[<Kegiatan 'Combining Web, Bot and AI in One App with Python'>]

In [69]:
kegiatan.peserta.append(tegar)

In [70]:
db.session.commit()

In [72]:
list_kegiatan('peserta','Combining Web, Bot and AI in One App with Python')

In [76]:
kegiatan = Kegiatan.query.filter_by(tema=tema).first()
for peserta in kegiatan.peserta:
    print(peserta.telegram)

tegarimansyah


In [10]:
nama = 'tegar'
email = 'egar@imansyah.com'
telegram = 'tegarimansyah'

tegar = Anggota(nama=nama, email=email, telegram=telegram)

In [11]:
savedb(tegar)

In [12]:
nama = 'udin'
email = 'udin@imansyah.com'
telegram = 'udinsedunia'

udin = Anggota(nama=nama, email=email, telegram=telegram)
savedb(udin)

In [35]:
post = db.session.query(Anggota).get(1)
post.ikut_serta

[<Kegiatan 'Combining Web, Bot and AI in One App with Python'>]

In [14]:
vdir(tegar)

['email',
 'github',
 'id',
 'ikut_serta',
 'key',
 'metadata',
 'nama',
 'query',
 'query_class',
 'tanggal_terdaftar',
 'telegram']

In [15]:
tegar.id

1

In [14]:
tema = 'Combining Web, Bot and AI in One App with Python'
pembicara = 'Tegar Imansyah'
tempat = '''Gedung Rektorat Lt 2
Ruangan Direktorat Sistem Informasi
Universitas Airlangga Surabaya (Kampus C)'''
waktu = tanggal('Apr 18 2018 6:00PM')
pendaftaran = tanggal('Apr 18 2016 5:59PM')

In [15]:
acara_unair = Kegiatan(tema = tema, pembicara = pembicara, tempat = tempat, waktu = waktu, pendaftaran = pendaftaran)

In [16]:
savedb(acara_unair)

In [20]:
vdir(acara_unair)

['id',
 'maps',
 'metadata',
 'pembicara',
 'pendaftaran',
 'peserta',
 'query',
 'query_class',
 'tema',
 'tempat',
 'waktu']

In [18]:
acara_unair.peserta.append(tegar)

In [19]:
db.session.commit()

In [20]:
for peserta in acara_unair.peserta:
    print(peserta.telegram)

tegarimansyah
udinsedunia


In [21]:
for kegiatan in tegar.ikut_serta:
    print(kegiatan)

<Kegiatan 'Combining Web, Bot and AI in One App with Python'>


In [30]:
Anggota.query.with_parent(self)

NameError: name 'self' is not defined

In [37]:
daftar_hadir

Table('daftar_hadir', MetaData(bind=None), Column('kegiatan', Integer(), ForeignKey('kegiatan.id'), table=<daftar_hadir>), Column('peserta', Integer(), ForeignKey('anggota.id'), table=<daftar_hadir>), Column('signup', DateTime(), table=<daftar_hadir>), Column('signin', DateTime(), table=<daftar_hadir>), schema=None)

In [15]:
db.session.rollback()

/home/tegar/.local/lib/python3.6/site-packages/sqlalchemy/orm/session.py:521: SAWarning: Session's state has been changed on a non-active transaction - this state will be discarded.
  "Session's state has been changed on "
